READ IMAGE AND CONVERT TO BINARY

In [955]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
import math
%matplotlib inline

# path
path = r'./IMG2.png'
# img = cv2.imread(path)
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# edges = cv2.Canny(img, 80, 120)


image = cv2.imread(path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
im_gray = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
thresh = 50
im_bw = cv2.threshold(im_gray, thresh, 255, cv2.THRESH_BINARY)[1]

# cv2.imwrite("./output.png", im_bw)


DETECT HORIZONTAL LINES AND CROP THE IMAGE IN SUBIMAGES FOR EVERY PLAYER 

In [956]:
# Detect horizontal lines
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50,1))
detect_horizontal = cv2.morphologyEx(im_bw, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
cnts = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

# sort by lenght
cnts = sorted(cnts, key=lambda item1: cv2.arcLength(item1, True), reverse=True)
cnts = cnts[0:13]

topImg = None
if((cv2.arcLength(cnts[0], True) - cv2.arcLength(cnts[1], True) > 200) or (cv2.arcLength(cnts[-1], True) - cv2.arcLength(cnts[-2], True) > -200)):
    c1Max = cnts[0][:, :, 1].max()
    c2Max = cnts[1][:, :, 1].max()
    topImg = image[c1Max:c2Max, 0:image.shape[1]]
    cnts = cnts[1:13]
else:
    cnts = cnts[0:12]
    

cnts = sorted(cnts, key=lambda item1: item1[0][0][1])
cropped_imgs = []
# print(cnts[5])
for i in range(0, len(cnts) - 1):
    if(i != 5):
        # get the lowest points of each contour
        c1Max = cnts[i][:, :, 1].max()
        c2Max = cnts[i + 1][:, :, 1].max()
        cropped_imgs.append(image[c1Max:c2Max, 0:image.shape[1]])
    # cv2.drawContours(image, [cnts[i]], 0, (0,255,0), 3)


# cv2.imwrite("./output.png", image)          
# print(len(cropped_imgs))
i = 0
for img in cropped_imgs:
    cv2.imwrite(f"./output{i}.png", img)
    i += 1




DETECT TEXT ON IMAGES

In [957]:
# from os import walk

# def compareItems(template):
#     template = cv2.cvtColor(template, cv2.COLOR_BGR2RGB)

#     files = next(walk('./tempItems'), (None, None, []))[2]  # [] if no file


In [958]:
class Player:
    def __init__(self):
        pass

    def toString(self):
        return f"lvl: {self.level} champion: {self.champion} username: {self.username} kda: {self.kda} dmg: {self.dmg} gold: {self.gold}"


def closestContour(item, cnt):
    x,y,w,h = cv2.boundingRect(item)
    x2, y2, w2, h2 = cv2.boundingRect(cnt)
    print(x, y, w, h)
    print(x2, y2, w2, h2)
    print(math.sqrt(pow(x - x2- pixelsCropped, 2) + pow((y + h) - y2, 2)))
    return math.sqrt(pow(x - x2- pixelsCropped, 2) + pow((y + h) - y2, 2))
    

In [959]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

def readText(image):
    image = 255 - image
    return pytesseract.image_to_string(image, config='--psm 7')
    


In [960]:
# # Importing the proper classes
# import os
# import pandas as pd
# import matplotlib.pyplot as plt
# from PIL import Image
# from tqdm import tqdm
# import numpy as np
# from annoy import AnnoyIndex
# from tqdm import tqdm
# from keras.preprocessing import image
# from keras.applications.vgg16 import VGG16, preprocess_input
# from keras.models import Model
# from keras.utils import image_utils
# import tensorflow as tf

# def read_data_from_folder(folder_path): # Enter the Single Folder Path/List of the Folders
#     image_path = []
#     for path in os.listdir(folder_path):
#         image_path.append(os.path.join(folder_path,path))
#     return image_path # Returning list of images

# class FeatureExtractor:
#     def __init__(self):
#         # Use VGG-16 as the architecture and ImageNet for the weight
#         base_model = VGG16(weights='imagenet')
#         # Customize the model to return features from fully-connected layer
#         self.model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
#     def extract(self, img):
#         # Resize the image
#         img = img.resize((224, 224))
#         # Convert the image color space
#         img = img.convert('RGB')
#         # Reformat the image
#         x = image_utils.img_to_array(img)
#         x = np.expand_dims(x, axis=0)
#         x = preprocess_input(x)
#         # Extract Features
#         feature = self.model.predict(x)[0]
#         return feature / np.linalg.norm(feature)
#     def get_feature(self,image_data:list):
#         self.image_data = image_data 
#         #fe = FeatureExtractor()
#         features = []
#         for img_path in tqdm(self.image_data): # Iterate through images 
#             # Extract Features
#             feature = self.extract(img=Image.open(img_path))
#             features.append(feature)
            
#         return features

# class image_comparator():
#     def __init__(self,image_list:list):
#         self.image_list = image_list
#         if 'meta-data-files' not in os.listdir():
#             os.makedirs("meta-data-files")
#         self.FE = FeatureExtractor()
#     def start_feature_extraction(self):
#         image_data = pd.DataFrame()
#         image_data['images_paths'] = self.image_list
#         f_data = self.FE.get_feature(self.image_list)
#         image_data['features'] = f_data
#         image_data = image_data.dropna().reset_index(drop=True)
#         image_data.to_pickle(os.path.join('meta-data-files/','image_data_features.pkl'))
#         print("Image Meta Information Saved: [meta-data-files/image_data_features.pkl]")
#         return image_data
#     def start_indexing(self,image_data):
#         self.image_data = image_data
#         f = len(image_data['features'][0]) # Length of item vector that will be indexed
#         t = AnnoyIndex(f, 'euclidean')
#         for i,v in tqdm(zip(self.image_data.index,image_data['features'])):
#             t.add_item(i, v)
#         t.build(100) # 100 trees
#         print("Saved the Indexed File:"+"[meta-data-files/image_features_vectors.ann]")
#         t.save(os.path.join('meta-data-files/','image_features_vectors.ann'))
#     def Start(self):
#         if len(os.listdir("meta-data-files/"))==0:
#             data = self.start_feature_extraction()
#             self.start_indexing(data)
#         else:
#             print("Metadata and Features are allready present, Do you want Extract Again? Enter yes or no")
#             flag  = str(input())
#             if flag.lower() == 'yes':
#                 data = self.start_feature_extraction()
#                 self.start_indexing(data)
#             else:
#                 print("Meta data allready Present, Please Apply Search!")
#                 print(os.listdir("meta-data-files/"))

# class SearchImage:
#     def __init__(self):
#         self.image_data = pd.read_pickle(os.path.join('meta-data-files/','image_data_features.pkl'))
#         self.f = len(self.image_data['features'][0])
#     def search_by_vector(self,v,n:int):
#         self.v = v # Feature Vector
#         self.n = n # number of output 
#         u = AnnoyIndex(self.f, 'euclidean')
#         u.load('meta-data-files/','image_features_vectors.ann') # super fast, will just mmap the file
#         index_list = u.get_nns_by_vector(self.v, self.n) # will find the 10 nearest neighbors
#         return dict(zip(index_list,self.image_data.iloc[index_list]['images_paths'].to_list()))
#     def get_query_vector(self,img:list):
#         fe = FeatureExtractor()
#         query_vector = fe.extract(img)
#         return query_vector
#     def get_most_similar_image(self,img:list):
#         query_vector = self.get_query_vector(img)
#         img_dict = self.search_by_vector(query_vector,1)
#         return img_dict


# image_list = read_data_from_folder('tempItems')
# # For Faster Serching we need to index Data first, After Indexing all the meta data stored on the local path
# image_comparator(image_list).Start()
# # for searching you need to give the image path and the number of similar image you want
# # image = cv2.imread('./outputItem.png')
# # SearchImage().get_most_similar_image(image)

In [961]:
i = 0
players = []

for img in cropped_imgs:
    originalImg = img
    player = Player()

    # DEFINE COLORS TO GET DATA
    lower = np.array([190, 190, 190])
    upper = np.array([246, 245, 245])

    # APPLY MASK AND CONVERT TO BINARY
    mask = cv2.inRange(img, lower, upper)

    thresh = cv2.adaptiveThreshold(mask,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)

    # Dilate to combine adjacent text contours
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 4))
    dilate = cv2.dilate(thresh, kernel, iterations=2)
    cv2.imwrite(f"./output_1.png", img)
    cv2.imwrite(f"./output_dilate1.png", dilate)
    

    # Find contours, highlight text areas, and extract ROIs
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    # cnts = filter(lambda item: cv2.contourArea(item) > 50, cnts)
    cnts = sorted(cnts, key=lambda item1: item1[0][0][0])

    # GET POSITION IMAGE
    x,y,w,h = cv2.boundingRect(cnts[0])
    player.position = img[y:y+h, x:x+w]
    cv2.imwrite(f"./output_position{i}.png", img[y:y+h, x:x+w])

    # cv2.drawContours(originalImg, cnts, 0, (0,255,0), 3)
    # cv2.imwrite(f"./output_contours0.png", originalImg)


    # GET LEVEL IMAGE
    x,y,w,h = cv2.boundingRect(cnts[1])
    player.level = readText(mask[y:y+h, x:x+w])
    cv2.imwrite(f"./output_level{i}.png", img[y:y+h, x:x+w])

    # CROP IMAGE
    img = img[:, x+w:dilate.shape[1]]
    cv2.imwrite(f"./output_cropped.png", img)

    pixelsCropped = x+w;

    # Dilate to combine adjacent text contours
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=3)
    cv2.imwrite(f"./output_dilate2.png", dilate)

    # Find contours, highlight text areas, and extract ROIs
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = filter(lambda item: cv2.contourArea(item) > 300, cnts)
    cnts = sorted(cnts, key=lambda item1: item1[0][0][0], reverse=True)

    # GET GOLD IMAGE
    x,y,w,h = cv2.boundingRect(cnts[0])
    player.gold = readText(mask[y:y+h, x:x+w])
    cv2.imwrite(f"./output_gold{i}.png", mask[y:y+h, x:x+w])

    # GET DMG IMAGE
    x,y,w,h = cv2.boundingRect(cnts[1])
    h = h if h <= ((mask.shape[0]/2) + 5) else (mask.shape[0]//2) + 5

    player.dmg = readText(mask[y:y+h, x:x+w])
    cv2.imwrite(f"./output_dmg{i}.png", mask[y:y+h, x:x+w])

    # GET KDA IMAGE
    x,y,w,h = cv2.boundingRect(cnts[2])
    player.kda = readText(img[y:y+h, x - pixelsCropped : x + w  - pixelsCropped])
    cv2.imwrite(f"./output_kda{i}.png", img[y:y+h, x - pixelsCropped : x + w  - pixelsCropped])

    cropRightX = x

    # DO THE PROCESS AGAIN TO GET THE CHAMPION NAME
    upper = np.array([170, 170, 170])
    lower = np.array([120, 120, 100])

    mask = cv2.inRange(img, lower, upper)
    thresh = cv2.adaptiveThreshold(mask,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)

    # Dilate to combine adjacent text contours
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    dilate = cv2.dilate(thresh, kernel, iterations=3)
    cv2.imwrite(f"./output_mask3.png", mask)
    cv2.imwrite(f"./output_dilate3.png", dilate)



    # Find contours, highlight text areas, and extract ROIs
    cnts2 = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts2 = cnts2[0] if len(cnts2) == 2 else cnts2[1]
    cnts2 = filter(lambda item: cv2.contourArea(item) > 150, cnts2)
    cnts2 = filter(lambda item: item[:, :, 0].min() < (dilate.shape[1] / 4) and item[:, :, 0].min() > (dilate.shape[1] / 15), cnts2)
    cnts2 = sorted(cnts2, key=lambda item1: item1[0][0][0])

    cnt = cnts2[0]
    x = 0
    while((x + 1) < len(cnts2) and (cnt[:, :, 1].max() - cnt[:, :, 1].min()) > 2 * (dilate.shape[0] / 3)):
        print(cnt[:, :, 0].min())
        x += 1
        cnt = cnts2[x]
    
    
    # GET CHAMPION IMAGE
    x,y,w,h = cv2.boundingRect(cnt)
    y = y if h <= ((mask.shape[0]/2) + 5) else (mask.shape[0]//2)
    h = h if h <= ((mask.shape[0]/2) + 5) else (mask.shape[0]//2)
    player.champion = readText(img[y:y+h, x:x+w])
    cv2.imwrite(f"./output_champion{i}.png", img[y:y+h, x:x+w])


    # GET THE COUNTOURN FROM CNTS CLOSEST (X, Y) TO cnt (WILL BE USERNAME)
    cnts = sorted(cnts, key=lambda item1: closestContour(item1, cnt))

    x,y,w,h = cv2.boundingRect(cnts[0])
    x2, y2, w2, h2 = cv2.boundingRect(cnt)

    # cv2.drawContours(originalImg, [cnts[0]], 0, (255,255,0), 3)
    # cv2.drawContours(originalImg, [cnt], 0, (255,255,0), 3)
    
    cv2.imwrite(f"./output_countorns.png", originalImg)
    # cv2.imwrite("./output_dilate.png", dilate)
    print(math.sqrt(pow(x - x2 - pixelsCropped, 2) + pow((y + h) - y2, 2)), (x - x2 - pixelsCropped))          

    if((math.sqrt(pow(x - x2 - pixelsCropped, 2) + pow((y + h) - y2, 2)) > 30) or (x - x2 - pixelsCropped) < -10):
        # YELLOW NAME
        upper = np.array([260, 190, 100])
        lower = np.array([200, 170, 5])

        mask = cv2.inRange(img, lower, upper)
        cv2.imwrite(f"./output_maskYellow.png", mask)
        thresh = cv2.adaptiveThreshold(mask,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)
        # Dilate to combine adjacent text contours
        dilate = cv2.dilate(thresh, kernel, iterations=5)
        cv2.imwrite(f"./output_dilateYellow.png", dilate)

        # Find contours, highlight text areas, and extract ROIs
        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        cnts = filter(lambda item: cv2.contourArea(item) > 300, cnts)
        cnts = sorted(cnts, key=lambda item1: closestContour(item1, cnts2[0]))
        x,y,w,h = cv2.boundingRect(cnts[0])
        player.username = readText(img[y:y+h, (x):(x+w)])
        cv2.imwrite(f"./output_username{i}.png", img[y:y+h, (x):(x+w)])

        cropLeftX = (x + w) if ((x + w) > (x2 + w2)) else (x2 + w2)

        player.user = True

    else:
        player.username = readText(img[y:y+h, (x - pixelsCropped):(x+w - pixelsCropped)])
        cv2.imwrite(f"./output_username{i}.png", img[y:y+h, (x - pixelsCropped):(x+w - pixelsCropped)])
        player.user = False

        cropLeftX = (x + w - pixelsCropped) if ((x + w - pixelsCropped) > (x2 + w2)) else (x2 + w2)

    img = img[2:img.shape[0] - 2, cropLeftX:cropRightX - pixelsCropped]




    # ITEMS && RUNES && SUMMS

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = 60
    im_bw = cv2.threshold(gray, thresh, 255, cv2.THRESH_BINARY)[1]
    blur = cv2.medianBlur(im_bw, 5)
    sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    sharpen = cv2.filter2D(blur, -1, sharpen_kernel)
    
    cv2.imwrite(f"./output_sharpen{i}.png", sharpen)

    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    # close = cv2.morphologyEx(blur, cv2.MORPH_CLOSE, kernel, iterations=1)
    # cv2.imwrite(f"./output_close{i}.png", close)

    # Find contours and filter using threshold area
    cnts = cv2.findContours(sharpen, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = filter(lambda item: cv2.contourArea(item) > 150, cnts)
    cnts = sorted(cnts, key=lambda item1: item1[0][0][0])

    # RUNES
    x, y, w, h = cv2.boundingRect(cnts[0])

    # 1 OR 2 SUMMS
    x2, y2, w2, h2 = cv2.boundingRect(cnts[1])
    runes = img[0:img.shape[0], int(x2 - 3 - img.shape[0] * 1.15):x2 - 3]
    player.runes = runes
    cv2.imwrite(f"./output_runes{i}.png", runes)

    x3, y3, w3, h3 = cv2.boundingRect(cnts[2])
    if((x2 + w2) < x3):
        #delete cnts[2]
        del cnts[2]

    summ1 = img[0:int(img.shape[0] / 2), x2:x2 + w2]
    player.summ1 = summ1
    summ2 = img[int(img.shape[0] / 2):img.shape[0], x2:x2 + w2]
    player.summ2 = summ2


    min_area = 100
    max_area = 1500
    image_number = 0
    items = []
    for c in cnts[2:]:
        area = cv2.contourArea(c)
        if area > min_area and area < max_area:
            x,y,w,h = cv2.boundingRect(c)
            item = img[y:y+h, x:x+w]
            cv2.imwrite(f"./outputItem_{i}_{image_number}.png", item)
            # compareItems(item)
            # item.append(item)
            # cv2.imwrite('ROI_{}.png'.format(image_number), ROI)
            # cv2.rectangle(img, (x, y), (x + w, y + h), (36,255,12), 2)
            image_number += 1
    player.items = items


    players.append(player)
    print(player.toString())

    # cv2.imwrite(f"./output{i}.png", )

    # x,y,w,h = cv2.boundingRect(cnts[1])
    # cv2.rectangle(mask, (x, y), (x + w, y + h), (36,255,12), 3)
    # cv2.imwrite(f"./{i}.png", mask)
    i += 1
    # result = img - masked



1325 4 83 30
153 32 99 23
1065.0018779326167
1190 4 82 30
153 32 99 23
930.002150535148
1046 4 98 30
153 32 99 23
786.0025445251434
713 15 25 29
153 32 99 23
453.1589125240725
256 5 79 28
153 32 99 23
4.123105625617661
165 0 22 18
153 32 99 23
96.0260381354974
89 14 42 34
153 32 99 23
171.74690681348528
53 14 29 29
153 32 99 23
207.2920644887305
4.123105625617661 -4
lvl: 1
 champion: Tryndamere
 username: arcadi3
 kda: 9/8/7
 dmg: 14.854
 gold: 13.246

1333 5 65 30
153 32 75 22
1073.0041938408256
1190 5 82 30
153 32 75 22
930.0048386970898
1035 5 123 30
153 32 75 22
775.0058064298615
256 5 92 27
153 32 75 22
4.0
89 15 41 34
153 32 75 22
171.8429515575195
57 21 25 24
153 32 75 22
203.4158302591025
4.0 -4
lvl: 1
 champion: Xin Zhao
 username: Rivalless
 kda: iil 7G)
 dmg: 13.157
 gold: 9786

1325 4 82 30
153 35 53 18
1065.0004694834645
1188 4 85 30
153 35 53 18
928.000538792947
1040 4 111 30
153 35 53 18
780.0006410253776
839 14 27 28
153 35 53 18
579.0423127889705
256 5 133 30
153 35 53

In [962]:
# import requests
# import json
# import os


# def downloadItems():
#     versions = json.loads(requests.get('https://ddragon.leagueoflegends.com/api/versions.json').text)
#     items = json.loads(requests.get(f'http://ddragon.leagueoflegends.com/cdn/{versions[0]}/data/en_US/item.json').text)

#     os.makedirs('./tempItems', exist_ok=True)
    
#     for item in items['data']:
#         tempItem = items['data'][item]
#         if(tempItem['gold']['purchasable']):
#             url = f'https://ddragon.leagueoflegends.com/cdn/{versions[0]}/img/item/{item}.png'
#             r = requests.get(url, allow_redirects=True)
#             open(f'./tempItems/{tempItem["name"]}.png', 'wb').write(r.content)

# # downloadItems()

